In [1]:
import os
import openai
from dotenv import load_dotenv
from tenacity import wait_chain

In [2]:
from dotenv import load_dotenv
import os

# Cargar variables desde .env
load_dotenv()

# Obtener el valor de la variable
mi_variable = os.getenv("OPENAI_API_KEY")
#print(mi_variable)  # Debería imprimir: Hola, soy una variable de entorno


In [3]:
openai.api_key = os.getenv("OPENAI_API_KEY")

## Archivo de leyes para el RAG


In [4]:
from langchain_community.document_loaders import PyPDFLoader

In [9]:
file_path= "codigopenalEC.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()
docs[0]

Document(metadata={'producer': 'HiQPdf 5.7', 'creator': 'PyPDF', 'creationdate': 'D:20140507084242', 'author': 'Fiel Web 13.0 de Ediciones Legales, 2013', 'keywords': 'CÓDIGO PENAL Ecuador Ley Norma', 'subject': 'CÓDIGO PENAL', 'title': 'CÓDIGO PENAL', 'source': 'codigopenalEC.pdf', 'total_pages': 171, 'page': 0, 'page_label': '1'}, page_content='CÓDIGO\xa0PENAL\nINTRODUCCIÓN\nEn\xa0la\xa0eterna\xa0e\xa0incesante\xa0lucha\xa0de\xa0la\xa0sociedad\xa0en\xa0pro\xa0de\xa0la\xa0perfección\xa0jurídica\xa0y\xa0el\xa0imperio\xa0de\xa0la\xa0ley,\xa0que\xa0se\xa0inicia\xa0en\xa0los\xa0ya\nlejanos\xa0tiempos\xa0primitivos\xa0de\xa0la\xa0venganza\xa0privada\xa0en\xa0su\xa0forma\xa0absoluta\xa0y\xa0de\xa0la\xa0venganza\xa0pública\xa0reglamentada\xa0como\npaliativo\xa0al\xa0exceso\xa0de\xa0la\xa0primera,\xa0y\xa0que\xa0deviene\xa0ininterrumpida\xa0hasta\xa0los\xa0días\xa0que\xa0decurren,\xa0preconizadores\xa0del\xa0respeto\xa0a\xa0la\ndignidad\xa0humana,\xa0el\xa0Derecho\xa0Penal\xa0\xado\xa0como\xa

In [6]:
pagina = loader.load()

In [13]:
import pprint

pprint.pp(docs[20].metadata)

{'producer': 'HiQPdf 5.7',
 'creator': 'PyPDF',
 'creationdate': 'D:20140507084242',
 'author': 'Fiel Web 13.0 de Ediciones Legales, 2013',
 'keywords': 'CÓDIGO PENAL Ecuador Ley Norma',
 'subject': 'CÓDIGO PENAL',
 'title': 'CÓDIGO PENAL',
 'source': 'codigopenalEC.pdf',
 'total_pages': 171,
 'page': 20,
 'page_label': '21'}


In [14]:
pagina

[Document(metadata={'producer': 'HiQPdf 5.7', 'creator': 'PyPDF', 'creationdate': 'D:20140507084242', 'author': 'Fiel Web 13.0 de Ediciones Legales, 2013', 'keywords': 'CÓDIGO PENAL Ecuador Ley Norma', 'subject': 'CÓDIGO PENAL', 'title': 'CÓDIGO PENAL', 'source': 'codigopenalEC.pdf', 'total_pages': 171, 'page': 0, 'page_label': '1'}, page_content='CÓDIGO\xa0PENAL\nINTRODUCCIÓN\nEn\xa0la\xa0eterna\xa0e\xa0incesante\xa0lucha\xa0de\xa0la\xa0sociedad\xa0en\xa0pro\xa0de\xa0la\xa0perfección\xa0jurídica\xa0y\xa0el\xa0imperio\xa0de\xa0la\xa0ley,\xa0que\xa0se\xa0inicia\xa0en\xa0los\xa0ya\nlejanos\xa0tiempos\xa0primitivos\xa0de\xa0la\xa0venganza\xa0privada\xa0en\xa0su\xa0forma\xa0absoluta\xa0y\xa0de\xa0la\xa0venganza\xa0pública\xa0reglamentada\xa0como\npaliativo\xa0al\xa0exceso\xa0de\xa0la\xa0primera,\xa0y\xa0que\xa0deviene\xa0ininterrumpida\xa0hasta\xa0los\xa0días\xa0que\xa0decurren,\xa0preconizadores\xa0del\xa0respeto\xa0a\xa0la\ndignidad\xa0humana,\xa0el\xa0Derecho\xa0Penal\xa0\xado\xa0como\x

In [18]:
import re
from langchain.text_splitter import TextSplitter

class RegexTextSplitter(TextSplitter):
    def __init__(self, pattern: str, keep_separator: bool = False):
        """
        Divisor de texto basado en expresiones regulares.

        :param pattern: Expresión regular para dividir el texto.
        :param keep_separator: Si se debe mantener el separador en los fragmentos.
        """
        self.pattern = pattern
        self.keep_separator = keep_separator
        super().__init__()

    def split_text(self, text: str):
        """Divide el texto según la expresión regular."""
        matches = list(re.finditer(self.pattern, text))
        chunks = []
        start = 0

        for match in matches:
            end = match.start()
            if end > start:
                chunk = text[start:end]
                if self.keep_separator:
                    chunk += match.group(0)  # Mantener el separador
                chunks.append(chunk.strip())
            start = match.end()

        # Agregar el último fragmento si queda texto restante
        if start < len(text):
            chunks.append(text[start:].strip())

        return chunks

# Expresión regular para capturar "Art. X.-", "Art. ... (X).-", "Art. X.X.-", etc.
splitter = RegexTextSplitter(
    pattern=r"(?=Art\.\s+((\d+|\...+)(\s*\(\d+\))?\.-|\d+\.\d+\.-))",
    keep_separator=True
)

# Verificar si pagina es una lista de objetos Document y extraer el texto
if isinstance(pagina, list):
    pagina = " ".join(doc.page_content for doc in pagina)

chunks = splitter.split_text(pagina)

# Mostrar los fragmentos obtenidos
for i, chunk in enumerate(chunks, 1):
    print(f"Fragmento {i}:")
    print(chunk)
    print("-" * 40)


Fragmento 1:
CÓDIGO PENAL
INTRODUCCIÓN
En la eterna e incesante lucha de la sociedad en pro de la perfección jurídica y el imperio de la ley, que se inicia en los ya
lejanos tiempos primitivos de la venganza privada en su forma absoluta y de la venganza pública reglamentada como
paliativo al exceso de la primera, y que deviene ininterrumpida hasta los días que decurren, preconizadores del respeto a la
dignidad humana, el Derecho Penal ­o como quiera que se le denomine por las tendencias científicas­ ha preocupado
apasionantemente a tratadistas y legisladores, juristas y magistrados, catedráticos e investigadores, por la razón elocuente
de que a la par que representa un poderoso instrumento jurídico para la defensa de la colectividad, incide con sus efectos,
la represión, la sanción o la pena, sobre los más preciados atributos y las más íntimas afecciones del hombre: su honor y su
fama, su nombre, su familia y su porvenir.
Transformada en efectiva la relación hipotética o preventiva que

In [19]:
len(chunk)

37965

In [20]:
# Guardar los fragmentos en un archivo .txt
with open('fragmentos.txt', 'w', encoding='utf-8') as file:
    for i, chunk in enumerate(chunks, 1):
        file.write(f"Fragmento {i}:\n")
        file.write(chunk + "\n")
        file.write("-" * 40 + "\n")


## Embeddings


In [4]:
from langchain_mistralai.embeddings import MistralAIEmbeddings


In [5]:
import os
load_dotenv()
api_keyMistral = os.getenv("MISTRAL_API_KEY")
hf_token = os.getenv("HF_TOKEN")


In [20]:

embedding_model = MistralAIEmbeddings(mistral_api_key=api_keyMistral)


/Users/victorcarrion/PycharmProjects/AgenteConversionalLeyes/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/victorcarrion/PycharmProjects/AgenteConversionalLeyes/.venv/lib/python3.11/site-packages/langchain_mistralai/embeddings.py:181: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [42]:
import time

chunk_embeddings = []

for chunk in chunks:
    success = False
    retries = 0
    max_retries = 5  # Número máximo de reintentos

    while not success and retries < max_retries:
        try:
            embedding = embedding_model.embed_query(chunk)
            chunk_embeddings.append(embedding)
            success = True  # Salir del bucle si se procesa correctamente
        except Exception as e:
            if "429" in str(e):  # Verifica si es error 429
                wait_time = 2 ** retries  # Backoff exponencial: 1, 2, 4, 8, ... segundos
                print(f"Error 429 recibido. Reintentando en {wait_time} segundos...")
                time.sleep(wait_time)
                retries += 1
            else:
                # Si el error no es 429, propágalo o maneja según convenga.
                raise e

    if not success:
        print("No se pudo obtener el embedding para este chunk después de varios reintentos.")


An error occurred with MistralAI: Client error '429 Too Many Requests' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429


Error 429 recibido. Reintentando en 1 segundos...


An error occurred with MistralAI: Client error '429 Too Many Requests' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429


Error 429 recibido. Reintentando en 1 segundos...


An error occurred with MistralAI: Client error '429 Too Many Requests' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429


...

Error 429 recibido. Reintentando en 1 segundos...


An error occurred with MistralAI: Client error '429 Too Many Requests' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429


Error 429 recibido. Reintentando en 1 segundos...


An error occurred with MistralAI: Server disconnected without sending a response.


RemoteProtocolError: Server disconnected without sending a response.

In [48]:
# Verificar qué fragmentos aún no tienen embeddings
fragments_with_embeddings = len(chunk_embeddings)  # Ya procesados
total_fragments = len(chunks)  # Total de fragmentos generados

print(f"Número total de fragmentos generados: {total_fragments}")
print(f"Número total de embeddings generados: {fragments_with_embeddings}")
print(f"⚠️ Faltan {total_fragments - fragments_with_embeddings} fragmentos por procesar.")


Número total de fragmentos generados: 1062
Número total de embeddings generados: 124
⚠️ Faltan 938 fragmentos por procesar.


In [49]:
# Solo incluir los fragmentos que aún no tienen embedding
chunks_missing_embeddings = chunks[len(chunk_embeddings):]  # Obtener los faltantes

print(f"Procesando {len(chunks_missing_embeddings)} fragmentos adicionales...")


Procesando 938 fragmentos adicionales...


In [50]:
import time

new_embeddings = []
max_retries = 5  # Máximo de intentos por fragmento

for chunk in chunks_missing_embeddings:
    success = False
    retries = 0

    while not success and retries < max_retries:
        try:
            embedding = embedding_model.embed_query(chunk)
            new_embeddings.append(embedding)
            success = True  # Salimos del bucle si se obtuvo el embedding
        except Exception as e:
            if "429" in str(e) or "RemoteProtocolError" in str(e):
                wait_time = 2 ** retries  # Backoff exponencial: 1, 2, 4, 8, 16 segundos...
                print(f"Error {e}. Reintentando en {wait_time} segundos...")
                time.sleep(wait_time)
                retries += 1
            else:
                print(f"Error inesperado: {e}")
                break  # Si el error no es manejable, salir del bucle

    if not success:
        print("⚠️ No se pudo obtener el embedding para un fragmento después de varios intentos.")


An error occurred with MistralAI: Client error '429 Too Many Requests' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429


Error Client error '429 Too Many Requests' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429. Reintentando en 1 segundos...


An error occurred with MistralAI: Client error '429 Too Many Requests' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429


Error Client error '429 Too Many Requests' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429. Reintentando en 2 segundos...


An error occurred with MistralAI: Client error '429 Too Many Requests' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429


...

Error Client error '429 Too Many Requests' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429. Reintentando en 1 segundos...


An error occurred with MistralAI: Client error '429 Too Many Requests' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429


Error Client error '429 Too Many Requests' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429. Reintentando en 2 segundos...


An error occurred with MistralAI: Client error '400 Bad Request' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/400


Error inesperado: Client error '400 Bad Request' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/400
⚠️ No se pudo obtener el embedding para un fragmento después de varios intentos.


In [51]:
chunk_embeddings.extend(new_embeddings)


In [52]:
if len(chunk_embeddings) == len(chunks):
    print("✅ Todos los fragmentos tienen su embedding correspondiente.")
else:
    print(f"⚠️ Aún faltan {len(chunks) - len(chunk_embeddings)} fragmentos.")


⚠️ Aún faltan 1 fragmentos.


In [55]:
# Obtener el fragmento problemático
fragment_faltante = chunks[len(chunk_embeddings)]

# Mostrar información del fragmento
print(f"Longitud del fragmento: {len(fragment_faltante)} caracteres")
print(f"Contenido del fragmento:\n{fragment_faltante[:500]}...")  # Mostramos los primeros 500 caracteres


Longitud del fragmento: 37965 caracteres
Contenido del fragmento:
Art. 4.- Añádase en el Libro II del Código Penal, a continuación del Título X "De los delitos contra la Propiedad", el
siguiente Título con el siguiente articulado: 
Título XI
Delitos de función de servidoras y servidores policiales y militares 
Capítulo I
De los delitos comunes de función 
Art. ... (602.3).­ Insubordinación.­ Será sancionado con prisión de tres meses a un año, la servidora o servidor militar o
policial que:
1. Rechazare, impidiere, o se resistiere violentamente al cumplimiento ...


In [56]:
if not fragment_faltante.strip():
    print("⚠️ El fragmento está vacío. No se requiere embedding.")
else:
    print("✅ El fragmento contiene texto. Intentaremos nuevamente procesarlo.")


✅ El fragmento contiene texto. Intentaremos nuevamente procesarlo.


In [57]:
max_tokens = 512  # Estimación de límite de tokens (ajustar si es necesario)

# Si el fragmento es muy grande, lo dividimos en partes más pequeñas
if len(fragment_faltante.split()) > max_tokens:
    print("⚠️ El fragmento es demasiado largo. Dividiéndolo en partes más pequeñas...")

    # Dividir el fragmento en segmentos más pequeños
    sub_chunks = [fragment_faltante[i:i+max_tokens] for i in range(0, len(fragment_faltante), max_tokens)]

    new_embeddings = []
    for sub_chunk in sub_chunks:
        try:
            embedding = embedding_model.embed_query(sub_chunk)
            new_embeddings.append(embedding)
        except Exception as e:
            print(f"❌ Error al procesar un subfragmento: {e}")

    # Unir embeddings promediando (opcional)
    if new_embeddings:
        final_embedding = [sum(x) / len(new_embeddings) for x in zip(*new_embeddings)]
        chunk_embeddings.append(final_embedding)
        print("✅ Fragmento procesado exitosamente después de dividirlo.")
    else:
        print("❌ No se pudo obtener embedding después de dividir el fragmento.")

else:
    # Si el fragmento no es demasiado largo, intentamos procesarlo normalmente
    try:
        embedding = embedding_model.embed_query(fragment_faltante)
        chunk_embeddings.append(embedding)
        print("✅ Fragmento final procesado correctamente.")
    except Exception as e:
        print(f"❌ Error inesperado: {e}")


⚠️ El fragmento es demasiado largo. Dividiéndolo en partes más pequeñas...


An error occurred with MistralAI: Client error '429 Too Many Requests' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429


❌ Error al procesar un subfragmento: Client error '429 Too Many Requests' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429


An error occurred with MistralAI: Client error '429 Too Many Requests' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429


❌ Error al procesar un subfragmento: Client error '429 Too Many Requests' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429


...

❌ Error al procesar un subfragmento: Client error '429 Too Many Requests' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429


An error occurred with MistralAI: Client error '429 Too Many Requests' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429


❌ Error al procesar un subfragmento: Client error '429 Too Many Requests' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429


An error occurred with MistralAI: Client error '429 Too Many Requests' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429


❌ Error al procesar un subfragmento: Client error '429 Too Many Requests' for url 'https://api.mistral.ai/v1/embeddings'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429
✅ Fragmento procesado exitosamente después de dividirlo.


In [58]:
if len(chunk_embeddings) == len(chunks):
    print("🎉 ¡Todos los fragmentos tienen su embedding correctamente!")
else:
    print(f"⚠️ Aún falta {len(chunks) - len(chunk_embeddings)} fragmento(s).")


🎉 ¡Todos los fragmentos tienen su embedding correctamente!


In [59]:
for i, (chunk, embedding) in enumerate(zip(chunks, chunk_embeddings), 1):
    print(f"Fragmento {i}: {chunk[:100]}...")  # Mostramos solo los primeros 100 caracteres
    print(f"Embedding {i} (primeros 5 valores): {embedding[:5]}")
    print("-" * 50)

Fragmento 1: CÓDIGO PENAL
INTRODUCCIÓN
En la eterna e incesante lucha de la sociedad en pro de la perfección jurí...
Embedding 1 (primeros 5 valores): [-0.016204833984375, 0.034210205078125, 0.01751708984375, -0.0224761962890625, 0.029327392578125]
--------------------------------------------------
Fragmento 2: Art. 1.- Leyes penales son todas las que contienen algún precepto sancionado con la amenaza de una p...
Embedding 2 (primeros 5 valores): [-0.0419921875, 0.03997802734375, 0.025146484375, -0.0025482177734375, 0.01715087890625]
--------------------------------------------------
Fragmento 3: Art. 2.- Nadie puede ser reprimido por un acto que no se halle expresamente declarado infracción por...
Embedding 3 (primeros 5 valores): [-0.0193328857421875, 0.03179931640625, 0.02349853515625, 0.0206298828125, 0.03521728515625]
--------------------------------------------------
Fragmento 4: Art. 3.- Se presume de derecho que las leyes penales son conocidas de todos aquellos sobre quienes i.

In [85]:
import pickle

# Guardar los embeddings en un archivo pickle
with open("embeddings.pkl", "wb") as f:
    pickle.dump(chunk_embeddings, f)

print("✅ Embeddings guardados en 'embeddings.pkl'.")


✅ Embeddings guardados en 'embeddings.pkl'.


In [86]:
import json

# Guardar los embeddings en un archivo JSON
with open("embeddings.json", "w") as f:
    json.dump(chunk_embeddings, f)

print("✅ Embeddings guardados en 'embeddings.json'.")


✅ Embeddings guardados en 'embeddings.json'.


In [4]:
import pickle

# Cargar los embeddings desde 'embeddings.pkl'
with open("embeddings.pkl", "rb") as f:
    chunk_embeddings = pickle.load(f)

print(f"✅ Embeddings cargados, total de embeddings: {len(chunk_embeddings)}.")


✅ Embeddings cargados, total de embeddings: 1062.


In [3]:
import json

# Cargar los embeddings desde 'embeddings.json'
with open("embeddings.json", "r") as f:
    chunk_embeddings = json.load(f)

print(f"✅ Embeddings cargados, total de embeddings: {len(chunk_embeddings)}.")


✅ Embeddings cargados, total de embeddings: 1062.


## Subir chroma

In [5]:
import shutil
import os

# Definir la ruta de almacenamiento
persist_directory = os.path.abspath("chroma")

# Eliminar cualquier carpeta anterior si aún existe
if os.path.exists(persist_directory):
    shutil.rmtree(persist_directory)
    print("✅ Base de datos eliminada.")

# Crear un nuevo directorio vacío para ChromaDB
os.makedirs(persist_directory, exist_ok=True)
print("✅ Nuevo directorio de persistencia creado.")


✅ Nuevo directorio de persistencia creado.


In [6]:
import chromadb
from chromadb.config import Settings

# Inicializar ChromaDB sin conflictos
chroma_client = chromadb.PersistentClient(
    path=persist_directory,
    settings=Settings(allow_reset=True)
)

# Crear una nueva colección
collection = chroma_client.get_or_create_collection(name="mis_embeddings")

print("✅ Base de datos ChromaDB creada desde cero y lista para usar.")


✅ Base de datos ChromaDB creada desde cero y lista para usar.


In [7]:
# Agregar embeddings a la colección ChromaDB
for i, embedding in enumerate(chunk_embeddings):
    # Suponiendo que el embedding es un vector y puede tener un "ID" o "metadata" asociado
    collection.add(
        documents=[f"Documento {i}"],  # Puedes poner un nombre o referencia a tu documento
        embeddings=[embedding],         # El embedding en sí
        metadatas=[{"source": f"Documento {i}"}],  # Metadatos (opcional)
        ids=[str(i)]  # Un identificador único para cada embedding
    )

print("✅ Embeddings añadidos correctamente a ChromaDB.")


✅ Embeddings añadidos correctamente a ChromaDB.


# Mistral

In [10]:
from langchain.vectorstores import Chroma
from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain_chroma import Chroma

persist_directory = "/Users/victorcarrion/PycharmProjects/AgenteConversionalLeyes/chroma"
# Cargar los embeddings desde ChromaDB
vector_store = Chroma(
    persist_directory=persist_directory,
    embedding_function=MistralAIEmbeddings(mistral_api_key=api_keyMistral)
)


In [11]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})  # Devuelve los 3 documentos más relevantes


In [16]:
from langchain_mistralai import ChatMistralAI


In [17]:
# Configurar el modelo de Mistral
llm = ChatMistralAI(
    mistral_api_key=api_keyMistral,
    model_name="mistral-tiny",  # Puedes usar "mistral-small" o "mistral-medium"
    temperature=0.2  # Controla la creatividad (0 = respuestas más precisas, 1 = más creativas)
)

In [18]:
response = llm.invoke("¿Que es el robo en Ecuador?")
print(response)



content='El robo en Ecuador se refiere a la acción de tomar o llevar por la fuerza o mediante la astucia algo que pertenece a otra persona sin su consentimiento. Puede ser un delito cometido contra personas o bienes, y puede tener diferentes formas, como el robo en el tránsito, el robo en el hogar, el robo en el trabajo, entre otros. El robo es un delito grave en Ecuador y se castiga con penas de prisión.' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 14, 'total_tokens': 139, 'completion_tokens': 125}, 'model': 'mistral-tiny', 'finish_reason': 'stop'} id='run-70a90c90-fb3c-4379-86f8-03573aae1bf3-0' usage_metadata={'input_tokens': 14, 'output_tokens': 125, 'total_tokens': 139}


## Integrar LLM con RAG

In [19]:
from langchain.chains import RetrievalQA

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"  # "stuff" une todo en un solo prompt, otras opciones: "map_reduce", "refine"
)


In [21]:
query = "¿Que es el robo en Ecuador??"
response = rag_chain.invoke({"query": query})  # Se pasa el query como diccionario

print("🔍 Respuesta del RAG:")
print(response)


🔍 Respuesta del RAG:
{'query': '¿Que es el robo en Ecuador??', 'result': 'El robo en Ecuador se refiere a la acción de tomar o hacerse con propiedades o bienes de otra persona sin su consentimiento y con la intención de mantenerlos para sí mismo o venderlos. Esto puede incluir robos a domicilios, robos en la calle, robos a negocios, robos a bancos y otros tipos de robos. El robo es un delito grave en Ecuador y se castiga con penas de prisión.'}


In [22]:
query = "¿Que aritucolos penan  el robo en Ecuador??"
response = rag_chain.invoke({"query": query})  # Se pasa el query como diccionario

print("🔍 Respuesta del RAG:")
print(response)

🔍 Respuesta del RAG:
{'query': '¿Que aritucolos penan  el robo en Ecuador??', 'result': 'En Ecuador, el robo es considerado un delito grave y está penalizado según la Ley Orgánica de Seguridad Ciudadana y la Ley Orgánica de Enjuiciamiento Penal. Los artículos penales específicos varían según el tipo y gravedad del robo cometido.\n\nPor ejemplo, el artículo 281 de la Ley Orgánica de Enjuiciamiento Penal establece que el robo cometido con violencia o amenaza de violencia es un delito de robo agravado, que se castiga con una pena de prisión de 4 a 12 años.\n\nEl artículo 282 de la misma ley establece que el robo cometido sin violencia o amenaza de violencia es un delito de robo simple, que se castiga con una pena de prisión de 1 a 4 años.\n\nEs importante tener en cuenta que estas son solo algunas de las penas que se pueden aplicar según el tipo y gravedad del robo cometido, y que la aplicación de la ley puede variar según el caso específico y las circunstancias en las que se cometió el d